In [7]:
import pickle as pkl
import random

In [4]:
with open('path-to-part.pkl', 'rb') as file:
    path_to_part = pkl.load(file)
with open('path-to-clientid.pkl', 'rb') as file:
    path_to_clientid = pkl.load(file)
with open('path-to-gender.pkl', 'rb') as file:
    path_to_gender = pkl.load(file)
with open('client-to-files.pkl', 'rb') as file:
    client_to_files = pkl.load(file)

In [5]:
all_files = list(path_to_part.keys())

In [6]:
all_files[0]

'common_voice_fa_20327670.wav'

In [11]:
random.choice(all_files)

'common_voice_fa_20995861.wav'

In [12]:
from collections import Counter

In [28]:
unique_men = set()
unique_women = set()
chosen_files = set()
num_man_files = 0
num_wman_files = 0
man_counter = Counter()
woman_counter = Counter()
iterations_counter = 0
while len(unique_men) < 2000 and len(unique_women) < 2000 and iterations_counter <= 1000000:
    iterations_counter += 1
    print(f'\r iter:{iterations_counter},{num_man_files} man files,{num_wman_files} woman files, {len(unique_men)} unique-men, {len(unique_women)} unique-women', end=' ')
    chosen_file = random.choice(all_files)
    if chosen_file in chosen_files:
        continue
    else:
        chosen_person = path_to_clientid[chosen_file]
        chosen_gender = path_to_gender[chosen_file]
        if chosen_gender == 'male':
            if len(unique_women) > 0 and len(unique_men) / len(unique_women) > 1.5:
                continue
            if (num_man_files > 0 and num_wman_files > 0) and num_man_files / num_wman_files > 1.5:
                continue
            if man_counter[chosen_person] <= 20:
                chosen_files.add(chosen_file)
                man_counter[chosen_person] += 1
                unique_men.add(chosen_person)
                num_man_files += 1
            else:
                continue
        elif chosen_gender == 'female':
            if (num_man_files > 0 and num_wman_files > 0) and num_wman_files / num_man_files > 1.5:
                continue
            if woman_counter[chosen_person] <= 20:
                chosen_files.add(chosen_file)
                woman_counter[chosen_person] += 1
                unique_women.add(chosen_person)
                num_wman_files += 1
            else:
                continue
        else:
            continue

 iter:36913,955 man files,1239 woman files, 277 unique-men, 184 unique-women

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 iter:427191,1024 man files,1537 woman files, 296 unique-men, 197 unique-women 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 iter:1000001,1024 man files,1537 woman files, 296 unique-men, 197 unique-women 

In [29]:
chosen_files_lst = list(chosen_files)

In [30]:
chosen_files_lst[:10]

['common_voice_fa_21580651.wav',
 'common_voice_fa_19582975.wav',
 'common_voice_fa_19439691.wav',
 'common_voice_fa_21157757.wav',
 'common_voice_fa_18740017.wav',
 'common_voice_fa_22947872.wav',
 'common_voice_fa_20977512.wav',
 'common_voice_fa_18242786.wav',
 'common_voice_fa_19210598.wav',
 'common_voice_fa_18740131.wav']

In [31]:
from tqdm import tqdm

In [42]:
file_usg_counter = Counter()
both_pair_count = 0
male_pair_count = 0
female_pair_count = 0
person_counter = Counter()
unique_persons = set()
negative_pairs = []
pair_count = Counter()
num_iterations = 0
while num_iterations <= 1000000:
    num_iterations += 1
    print(f'\r iter:{num_iterations}, {both_pair_count} both pairs, {male_pair_count} male pairs, {female_pair_count} female pairs, {len(unique_persons)} unique persons', end=' ')
    first_file = random.choice(chosen_files_lst)
    first_person = path_to_clientid[first_file]
    first_gender = path_to_gender[first_file]
    second_file = random.choice(chosen_files_lst)
    second_person = path_to_clientid[second_file]
    second_gender = path_to_gender[second_file]
    if person_counter[first_person] > 50 or person_counter[second_person] > 50:
        continue
    if pair_count[first_file+'-'+second_file] > 0:
        continue
    if person_counter[first_person] <= 50 and person_counter[second_person] <= 50:
        if (first_gender == 'male' and second_gender == 'female') or (first_gender == 'female' and second_gender == 'male'):
            curr_pair_tag = 'both'
        elif first_gender == 'male' and second_gender == 'male':
            curr_pair_tag = 'male'
        elif first_gender == 'female' and second_gender == 'female':
            curr_pair_tag = 'female'
        if curr_pair_tag == 'both':
            if (both_pair_count > 0 and male_pair_count > 0 and female_pair_count > 0) and (both_pair_count / male_pair_count > 1.5 or both_pair_count / female_pair_count > 1.5):
                continue
            else:
                negative_pairs.append([first_file, second_file])
                both_pair_count += 1
                unique_persons.add(first_person)
                unique_persons.add(second_person)
                person_counter[first_person] += 1
                person_counter[second_person] += 1
        elif curr_pair_tag == 'male':
            if (both_pair_count > 0 and male_pair_count > 0 and female_pair_count > 0) and (male_pair_count / both_pair_count > 1.5 or male_pair_count / female_pair_count > 1.5):
                continue
            else:
                negative_pairs.append([first_file, second_file])
                male_pair_count += 1
                unique_persons.add(first_person)
                unique_persons.add(second_person)
                person_counter[first_person] += 1
                person_counter[second_person] += 1
        elif curr_pair_tag == 'female':
            if (both_pair_count > 0 and male_pair_count > 0 and female_pair_count > 0) and (female_pair_count / both_pair_count > 1.5 or female_pair_count / male_pair_count > 1.5):
                continue
            else:
                negative_pairs.append([first_file, second_file])
                female_pair_count += 1
                unique_persons.add(first_person)
                unique_persons.add(second_person)
                person_counter[first_person] += 1
                person_counter[second_person] += 1

 iter:82472, 3449 both pairs, 2439 male pairs, 2506 female pairs, 493 unique persons 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 iter:119253, 3766 both pairs, 2884 male pairs, 2568 female pairs, 493 unique persons 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 iter:278698, 3997 both pairs, 3928 male pairs, 2664 female pairs, 493 unique persons

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 iter:1000001, 4163 both pairs, 4163 male pairs, 2775 female pairs, 493 unique persons 

In [45]:
len(negative_pairs)

11101

In [46]:
with open('negative-pairs.pkl', 'wb') as file:
    pkl.dump(negative_pairs, file)

In [47]:
negative_pairs[100]

['common_voice_fa_19410043.wav', 'common_voice_fa_18218166.wav']

In [49]:
clients_lst = list(set(list(path_to_clientid.values())))

In [51]:
clients_lst = clients_lst[1:]

In [52]:
clients_lst[:10]

['c1b79072d7ad0f8585bf7183fa9f8dd8fbe9ff3d61265463bb77d873141fa3459a3567782e8606ebe13613cdd7dabc59ee2b5c2a939670c4b77904b785b6a3c9',
 'a42a528fa4a6520904e07713c40124c9260b21c87c5da7dfa78661c924e299826004f8ce6cc4fde1fd19a6baee16f05296e272cb69522cc33de04b827abf3931',
 '26bb5fca874f9b113b0098d25616a73fc989f5ed3c9a18bd6e77849c463997e03fde3147cb14b7e9ae47e38a7d4163ed1b5e25730bd5e07d70f9d27c70c194cb',
 '8058b06b9c7de74481addbd96a5fcb3b71988dfd89728fa929c74e69d31a02fa6decf8cb868ff6c86f24a46e152b53e6e24ccf0b2c1900135310d03d663065ed',
 'ce112ecbfd879414764152344be3f623156838e7e23103621ca99b0e2d2cf0cd3f372d1618056961fbb6d210cc37a6b9993b2035c95716b9fffd3a04290d1ab7',
 'dd63db339548df86826bd43b6a02a59978f58f4f2e4c4a4f815c5e5d6a0c3cd4ab534dbcbb995e1a6e1c555d0eea1082b3f3ed6760ee45fca5a4ba87ae69d9e0',
 '7666b2aeeca31959e4b72ee74d34f8d3fae1cbf24d9f07f6ecbe1ab4bfc854f9b9738949fb5433233f7772feeee39c20137f8d2cbe249af0af8e7ed465449005',
 '921dc086b4b410e60fa181c8ab6db81411940e00eda90c5ab1614d0dae5ef75c230

In [55]:
del path_to_clientid[nan]

NameError: name 'nan' is not defined

In [69]:
person_to_gender = {}
for (p, g) in list(path_to_gender.items()):
    if type(p) == 'str' and type(g) == 'str':
#     print(p)
#     print(g)
        client = path_to_clientid[p]
        person_to_gender[client] = g

In [71]:
path_to_gender[client_to_files['7666b2aeeca31959e4b72ee74d34f8d3fae1cbf24d9f07f6ecbe1ab4bfc854f9b9738949fb5433233f7772feeee39c20137f8d2cbe249af0af8e7ed465449005'][0]]

nan

In [80]:
num_iterations = 0
unique_men = set()
unique_women = set()
pair_counter = Counter()
men_counter = Counter()
women_counter = Counter()
positive_pairs = []
while num_iterations <= 1000000:
    num_iterations += 1
    print(f'\r iter:{num_iterations}, {len(unique_men)} unique men, {len(unique_women)} unique women, {len(pairs)} pairs', end=' ')
    person = random.choice(clients_lst)
    if path_to_gender[client_to_files[person][0]] not in {'male', 'female'}:
        continue
    gender = path_to_gender[client_to_files[person][0]]
    curr_files_lst = client_to_files[person]
    if len(curr_files_lst) < 2:
        continue
    curr_pair = random.sample(curr_files_lst, 2)
    if pair_counter[curr_pair[0]+'-'+curr_pair[1]] > 0:
        continue
    if (gender == 'male' and men_counter[person] > 50) or (gender=='female' and women_counter[person] > 50):
        continue
    if len(unique_women)>0 and (len(unique_men) / len(unique_women) > 1.5) and gender == 'male':
        continue
    if gender == 'male':
        unique_men.add(person)
        men_counter[person] += 1
    elif gender == 'female':
        unique_women.add(person)
        women_counter[person] += 1
    positive_pairs.append(curr_pair)
    pair_counter[curr_pair[0]+'-'+curr_pair[1]] += 1

 iter:79990, 329 unique men, 219 unique women, 11190 pairs

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 iter:191606, 329 unique men, 219 unique women, 11190 pairs 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 iter:257926, 329 unique men, 219 unique women, 11190 pairs 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 iter:316221, 329 unique men, 219 unique women, 11190 pairs

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 iter:1000001, 329 unique men, 219 unique women, 11190 pairs 

In [81]:
with open('positive-pairs.pkl', 'wb') as file:
    pkl.dump(positive_pairs, file)

In [83]:
positive_pairs[100]

['common_voice_fa_18582526.wav', 'common_voice_fa_18582672.wav']